In [1]:
import carla

In [3]:
client = carla.Client('localhost', 2000)
world = client.load_world('Town01')

In [4]:
spawn_points = world.get_map().get_spawn_points()

In [5]:
vehicle_bp = world.get_blueprint_library().filter('*charger*')
start_point = spawn_points[0]
vehicle = world.try_spawn_actor(vehicle_bp[1], start_point)

In [6]:
vehicle_pos = vehicle.get_transform()
print("Position:",vehicle_pos)

Position: Transform(Location(x=335.489990, y=273.743317, z=-0.000077), Rotation(pitch=0.000000, yaw=90.000046, roll=0.000000))


In [7]:
import time
import cv2 as cv
import numpy as np
import math

In [8]:
# setting up camera
# offset wrt car
camera_z = 3
camera_x = -5
camera_y = 0

camera_bp = world.get_blueprint_library().find('sensor.camera.rgb')
camera_bp.set_attribute('image_size_x', '640')
camera_bp.set_attribute('image_size_y', '360')

camera_transform = carla.Transform(carla.Location(z=camera_z, x=camera_x, y=camera_y))
camera = world.spawn_actor(camera_bp,camera_transform,attach_to=vehicle)

In [9]:
# continous stream of images updated to data_dict
def camera_callback(image,data_dict):
    data_dict['image'] = np.reshape(np.copy(image.raw_data),(image.height,image.width,4))
    
image_w = camera_bp.get_attribute('image_size_x').as_int()
image_h = camera_bp.get_attribute('image_size_y').as_int()
camera_data = {'image':np.zeros((image_h,image_w,4))}

camera.listen(lambda image: camera_callback(image,camera_data))

In [10]:
speed_limit = 30
speed_soft_max = 25

font = cv.FONT_HERSHEY_SIMPLEX

org = (30,30)
org2 = (30,50)
org3 = (30,70)
fontScale = 0.5
color = (255,255,255)
thickness = 1

def maintain_speed(cur_speed):
    if cur_speed > speed_limit:
        return 0
    elif cur_speed < speed_soft_max:
        return 0.8
    else:
        return 0.2

In [13]:
cv.namedWindow('RGB Camera', cv.WINDOW_AUTOSIZE)
cv.imshow('RGB Camera', camera_data['image'])

quit = False

while True:
    world.tick()
    if cv.waitKey(1) == ord('q'):
        quit = True
        break
    image = camera_data['image']
    steering_angle = 0
    cur_velocity = vehicle.get_velocity()
    # convert velocity to scalar speed
    cur_speed = round(3.6* math.sqrt(cur_velocity.x**2+cur_velocity.y**2+cur_velocity.z**2),0)
    
    image=cv.putText(image, 'Speed:' +str(int(cur_speed))+'kmph',
                     org2,font,fontScale,color,thickness,cv.LINE_AA)
    throttle = maintain_speed(cur_speed)
    vehicle.apply_control(carla.VehicleControl(throttle=throttle,steer=steering_angle))
    cv.imshow('RGB Camera', image)
    
cv.destroyAllWindows()
camera.stop()
for actor in world.get_actors().filter('*vehicle*'):
    actor.destroy()
for sensor in world.get_actors().filter('*sensor*'):
    sensor.destroy()